In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense, Options
import numpy as np
import sys
import gamspy as gp
import math
import matplotlib.pyplot as plt
import pickle
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle

In [4]:
import numpy as np
import pandas as pd

# Step 1: Define parameters
num_stocks = 30          # Total number of stocks
num_increasing = 25      # Number of stocks with increasing prices
num_decreasing = 5      # Number of stocks with decreasing prices
num_months = 13          # Number of months (time periods)
initial_price = 100      # Initial price for all stocks

# Ensure the total number of stocks matches the sum of increasing and decreasing stocks
assert num_increasing + num_decreasing == num_stocks, "The sum of increasing and decreasing stocks must equal the total number of stocks."

# Step 2: Generate fake price data
price_array = np.zeros((num_stocks, num_months))

# Generate increasing prices for the specified number of stocks
for i in range(num_increasing):
    # Simulate an increasing trend by adding a random increment each month
    price_array[i] = initial_price + np.cumsum(np.random.uniform(1, 5, num_months))

# Generate decreasing prices for the specified number of stocks
for i in range(num_increasing, num_increasing + num_decreasing):
    # Simulate a decreasing trend by subtracting a random decrement each month
    price_array[i] = initial_price - np.cumsum(np.random.uniform(1, 5, num_months))

# Ensure no negative prices (optional, depending on the simulation)
price_array = np.maximum(price_array, 0)

# Step 3: Convert the price array into a DataFrame for better visualization
tickers = [f"Stock_{i+1}" for i in range(num_stocks)]
months = [f"Month_{i+1}" for i in range(num_months)]

price_df = pd.DataFrame(price_array, index=tickers, columns=months)

# Step 4: Print the resulting array and DataFrame
print("Price array shape:", price_array.shape)  # Should be (num_stocks, num_months)
print("Fake Price Array:")
print(price_array)

print("\nFake Price DataFrame:")
print(price_df)

# Optional: Save the fake price data to a CSV file
price_df.to_csv('fake_stock_prices.csv')

Price array shape: (30, 13)
Fake Price Array:
[[104.0836845  108.78444936 113.19073022 116.48144756 119.92231342
  123.08878449 126.26594971 130.38942652 131.70279885 136.49459019
  139.07066559 142.55755562 147.29017978]
 [101.81376842 103.42454393 107.84967761 109.80119173 112.27879976
  115.26272782 116.83826626 120.72995661 123.04579908 125.71986069
  129.34944053 132.93063657 135.35864371]
 [101.78970446 104.62773353 107.79669639 111.29840424 114.77704777
  118.59955639 119.65284967 123.90222669 126.77683963 131.10155568
  134.29881998 138.60044155 140.59902301]
 [103.77225121 106.5020821  109.32036327 113.77449906 117.76722543
  119.21563902 123.21443452 127.47961657 128.74199731 131.55831401
  132.67775851 134.78508408 138.56978907]
 [102.1402847  103.77418268 108.23791856 111.32439149 114.07184563
  118.08079052 122.58294115 126.44009615 129.7233318  131.0433712
  132.07553402 133.56719663 134.74299064]
 [102.6426475  105.56625282 108.06375716 111.4286766  115.51159604
  116.76

In [32]:

# Portfolio Configuration Parameters
max_exposure = 0.15  # Maximum exposure per stock
buy_exposure = 0.10  # Maximum buying limit as a fraction of NAV
max_return = 1.80  # Cap on maximum return per stock
PT = 0.50  # Portfolio turnover rate
cash_min = 0.001  # Minimum cash position as a fraction of NAV
bfee = 1.02  # Buy fee multiplier
sfee = 0.98  # Sell fee multiplier
min_buy = 0.001  # Minimum purchase amount
expected_return = 1.30  # Target return for the portfolio
init_cash = 1000000  # Initial cash balance
eps = 0.0001
M = 100000000

buy_minimum = 50000


# Define the model container
m = Container()

# Sets
time_index = 13  # Total number of time periods
stocks = Set(container=m, name="stocks", records=tickers)  # Set of stocks
time = Set(container=m, name="time", records=np.arange(0, time_index, 1))  # Time periods
subtime = Set(container=m, domain=time, records=np.arange(1, time_index, 1))  # Excludes first period
subtime2 = Set(container=m, domain=time, records=np.arange(2, time_index, 1))  # Excludes first two periods

# Data: Stock prices indexed by stocks and time
prices = Parameter(
    container=m,
    name="prices",
    domain=[stocks, time],
    records=price_array,
)

# Variables
C = Variable(container=m, name="C", domain=time, type="Positive")  # Cash balance
V = Variable(container=m, name="V", domain=time, type="Positive")  # Net Asset Value
x = Variable(container=m, name="x", domain=[stocks, time], type="Positive")  # Holdings
x.fx[stocks, '0'] = 0  # No initial holdings

b = Variable(container=m, name="b", domain=[stocks, time], type="Positive")  # Stocks bought
bp = Variable(container=m, name="bp", domain=[stocks, time], type="Positive")  # Stocks bought value

sp = Variable(container=m, name="sp", domain=[stocks, time], type="Positive")
b.fx[stocks, '0'] = 0  # No initial buying

s = Variable(container=m, name="s", domain=[stocks, time], type="Positive")  # Stocks sold

Z = Variable(container=m, name="Z", type="free")  # Objective variable
Sr = Variable(container=m, name="Sr", domain=[stocks], type="Positive")  # Total sales
Br = Variable(container=m, name="Br", domain=[stocks], type="Positive")  # Total purchases



gamma_s = Variable(container=m, name="gs", domain=[stocks,time], type="Integer")  # Total purchases
gamma_b = Variable(container=m, name="gb", domain=[stocks,time], type="Integer")  # Total purchases




#Br.lo[stocks] = 20000


profit = Variable(container=m, name="profit", domain=[stocks], type="Positive")  # Profit per stock

## Binary variable if stock i is a winner
omega = Variable(container=m, name="omega", domain=[stocks], type="Binary")

## Binary variable if stock i was a trade
omega_trades = Variable(container=m, name="omega_stock", domain=[stocks], type="Binary")


## winner_determination of stock i
buy_or_sell = Equation(
    m,
    name="buy_or_sell",
    domain=[stocks,time],
    description="Determines whether the stock is sold or bought at time t"
)
buy_or_sell[stocks,time] = (
   1 >= gamma_s[stocks,time]  + gamma_b[stocks,time]
)



## winner_determination of stock i
stocks_bought_value = Equation(
    m,
    name="stocks_bought_value",
    domain=[stocks,time],
    description="Determines the value of the stocks bought at time t"
)
stocks_bought_value[stocks,time] = (
   bp[stocks,time] == b[stocks,time]*prices[stocks,time]
)


## winner_determination of stock i
stocks_sold_value = Equation(
    m,
    name="stocks_sold_value",
    domain=[stocks,time],
    description="Determines the value of the stocks sold at time t"
)
stocks_sold_value[stocks,time] = (
   sp[stocks,time] == s[stocks,time]*prices[stocks,time]
)


## winner_determination of stock i
winner_determination = Equation(
    m,
    name="winner_determination",
    domain=[stocks],
    description="Determines if investing in stock i was a winner through the lifetime of the fund"
)
winner_determination[stocks] = (
    (Sr[stocks] + x[stocks, str(time_index-1)]*prices[stocks, str(time_index-1)]) >= Br[stocks] + eps*omega[stocks] - M*(1-omega[stocks])
)



## loser_determination of stock i
loser_determination = Equation(
    m,
    name="loser_determination",
    domain=[stocks],
    description="Determines if investing in stock i was a loser through the lifetime of the fund"
)
loser_determination[stocks] = (
    (Sr[stocks] + x[stocks, str(time_index-1)]*prices[stocks, str(time_index-1)]) -  Br[stocks]  <=  -eps*(1-omega[stocks]) + M*omega[stocks]
)


## Determination of omega_trades for stock i
omega_trades_determination = Equation(
    m,
    name="omega_trades_determination",
    domain=[stocks],
    description="Determines if Br[stocks] > 1 and sets omega_trades to 1 in that case"
)
omega_trades_determination[stocks] = (
    Br[stocks] <= 1000000 * omega_trades[stocks]
)



## Determination of omega_trades for stock i
omega_trades_max = Equation(
    m,
    name="omega_trades_max",
    domain=[stocks],
    description="Determines the maximum number of stocks that can be bought"
)
omega_trades_max[stocks] = (
    Br[stocks] >= buy_minimum * omega_trades[stocks]
)



# Equations: Total sales and purchases
total_sales = Equation(
    m,
    name="total_sales",
    domain=[stocks],
    description="Total revenue from selling stock",
)
total_sales[stocks] = Sr[stocks] == Sum(time, prices[stocks, time] * s[stocks, time])

total_purchases = Equation(
    m,
    name="total_purchases",
    domain=[stocks],
    description="Total cost from buying stock",
)
total_purchases[stocks] = Br[stocks] == Sum(time, prices[stocks, time] * b[stocks, time])

# Profit calculation
pnl = Equation(
    m,
    name="pnl",
    domain=[stocks],
    description="Calculate profit for each stock",
)
pnl[stocks] = (
    profit[stocks] ==
    (Sr[stocks] + x[stocks, str(time_index - 1)] * prices[stocks, str(time_index - 1)]) - Br[stocks]
)

# Turnover constraint
portfolio_turnover = Equation(
    m,
    name="portfolio_turnover",
    description="Portfolio turnover calculation",
)
portfolio_turnover[...] = (
    PT * (V["1"] + V[str(time_index - 1)]) ==
    Sum(stocks, Sum(subtime2, b[stocks, subtime2] * prices[stocks, subtime2])) * 2
)

# Max returns per stock
max_returns = Equation(
    m,
    name="max_returns",
    domain=[stocks],
    description="Cap on maximum returns per stock",
)
max_returns[stocks] = (
    (Sr[stocks] + x[stocks, str(time_index - 1)] * prices[stocks, str(time_index - 1)]) <=
    Br[stocks] * max_return
)

# Holdings and cash balance equations
holdings_balance = Equation(
    m,
    name="holdings_balance",
    domain=[stocks, time],
    description="Holdings balance over time",
)
holdings_balance[stocks, subtime] = (
    x[stocks, subtime] == x[stocks, subtime - 1] + b[stocks, subtime] - s[stocks, subtime]
)

holdings_balance_initial = Equation(
    m,
    name="holdings_balance_initial",
    domain=[stocks],
    description="Initial stock holdings",
)
holdings_balance_initial[stocks] = x[stocks, "0"] == 0

cash_balance = Equation(
    m,
    name="cash_balance",
    domain=[time],
    description="Cash balance at each time period",
)
cash_balance[time] = (
    C[time] == C[time - 1] -
    Sum(stocks, bfee * bp[stocks, time]*gamma_b) +
    Sum(stocks, sfee * sp[stocks, time]*gamma_s)
)

cash_balance_initial = Equation(
    m,
    name="cash_balance_initial",
    description="Initial cash balance",
)
cash_balance_initial[...] = C["0"] == init_cash

# NAV calculation
nav = Equation(
    m,
    name="nav",
    domain=[time],
    description="Net Asset Value calculation",
)
nav[time] = V[time] == C[time] + Sum(stocks, prices[stocks, time] * x[stocks, time])

# Risk constraints
risk_constraint = Equation(
    m,
    name="risk_constraint",
    domain=[stocks, time],
    description="Limit position exposure",
)
risk_constraint[stocks, time] = max_exposure * V[time] >= prices[stocks, time] * x[stocks, time]

buy_risk_constraint = Equation(
    m,
    name="buy_risk_constraint",
    domain=[stocks, time],
    description="Limit buying exposure",
)
buy_risk_constraint[stocks, time] = buy_exposure * V[time] >= prices[stocks, time] * b[stocks, time]

# Deviation and objective function
Z_plus = Variable(container=m, name="Z_plus", type="Positive")
Z_minus = Variable(container=m, name="Z_minus", type="Positive")
constraint_deviation = Equation(
    container=m,
    name="constraint_deviation",
)
constraint_deviation[...] = (
    Sum(stocks, omega[stocks]) == Z_plus #- Z_minus
)


# Deviation and objective function
portfolio_returns = Equation(
    container=m,
    name="portfolio_returns",
)
portfolio_returns[...] = (
    V[str(time_index - 1)] - expected_return * init_cash  == 0
)




obj_function = Equation(
    container=m,
    name="obj_function",
)
obj_function[...] = Z == Z_plus 

# Model definition
b1 = Model(
    container=m,
    name="b1",
    equations=m.getEquations(),
    problem="MIP",
    sense=Sense.MIN,
    objective=Z,
)

# Solve the model
gdx_path = m.gdxOutputPath()
b1.solve(
    output=sys.stdout,
    options=Options(report_solution=1),
    solver_options={
        "reslim": "100",
         "SolnPoolReplace": 2,
         "SolnPoolIntensity": 4,
         "SolnPoolPop": 2,
         "PopulateLim": 1000,
         "solnpoolmerge": "mysol.gdx",
    }
)


print(omega_trades.records.level.sum())

--- Job _ffcee3d2-c147-4ab8-95e5-1d51dfbebe2e.gms Start 02/04/25 19:24:46 48.6.1 67fbb04b DAX-DAC arm 64bit/macOS
--- Applying:
    /Users/sarangbalan/miniconda3/envs/py310/lib/python3.10/site-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    MIP CPLEX
    Input /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpdfewcv3t/_ffcee3d2-c147-4ab8-95e5-1d51dfbebe2e.gms
    Output /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpdfewcv3t/_ffcee3d2-c147-4ab8-95e5-1d51dfbebe2e.lst
    ScrDir /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpdfewcv3t/tmpv8b_1ow8/
    SysDir /Users/sarangbalan/miniconda3/envs/py310/lib/python3.10/site-packages/gamspy_base/
    LogOption 3
    Trace /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpdfewcv3t/_ffcee3d2-c147-4ab8-95e5-1d51dfbebe2e.txt
    License "/Users/sarangbalan/Library/Application Support/GAMSPy/gamspy_license.txt"
    OptFile 1
    OptDir /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpdfewcv3t/
    LimRow 0
    LimCo

GamspyException: Return code 2. There was a compilation error. Check /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpdfewcv3t/_ffcee3d2-c147-4ab8-95e5-1d51dfbebe2e.lst for more information.

=============
Error Summary
=============
 412  solve b1 using MIP MIN Z;
****                          $56,256
**** LINE      4 INPUT       /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpdfewcv3t/_ffcee3d2-c147-4ab8-95e5-1d51dfbebe2e.gms
****  56  Endogenous operands for * not allowed in linear models
**** 256  Error(s) in analyzing solve statement.
**** The following MIP errors were detected in model b1:
****  56 equation cash_balance.. VAR operands for *
****  56 equation cash_balance.. VAR operands for *
 413   

**** 2 ERROR(S)   0 WARNING(S)


COMPILATION TIME     =        0.000 SECONDS      4 MB  48.6.1 67fbb04b DAX-DAC


USER: Sarang Balan                                   G241122+0003Ac-GEN
      sarang.balan.20@ucl.ac.uk                               GPA102651
      License for t

In [27]:
sp.records[sp.records.time == "1"]['level'].sum()

np.float64(567129.6189986765)

In [28]:
bp.records[bp.records.time == "1"]['level'].sum()

np.float64(1525281.3986457868)